# Step 0: Preparations

In [1]:
import pickle
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np

In [2]:
!git clone https://github.com/MahdiTheGreat/Intro-to-language-modeling.git

Cloning into 'Intro-to-language-modeling'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 127 (delta 68), reused 2 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (127/127), 34.79 MiB | 14.05 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [3]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
! pip install datasets

In [5]:
# Data
dataset='/content/Intro-to-language-modeling/pa4'
zip_file = f"{dataset}.zip"
! unzip -q {zip_file}
! rm {zip_file}

# Step 1: Full fine tuning

### Preprocessing

In [6]:
from datasets import load_dataset
imdb_dataset = load_dataset('csv', data_files = {'train': '/content/train.csv', 'eval': '/content/eval.csv'})

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize_helper(batch):
    return tokenizer(batch['review'], padding=True, truncation=True)
tokenized_imdb_dataset = imdb_dataset.map(tokenize_helper, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

### Creating classifier

In [13]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Sanity check
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


### Counting the number of trainable paramters

In [15]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [16]:
# Sanity check
count_trainable_parameters(model)

66955010

### Preparing for training

In [24]:
from transformers import TrainingArguments
training_arguments = TrainingArguments(output_dir = 'out', num_train_epochs=10, eval_strategy='epoch')

In [18]:
import evaluate

accuracy_scorer = evaluate.load('accuracy')

def evaluation_helper(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy_scorer.compute(predictions=predictions, references=labels)

### Training the model

In [25]:
from transformers import Trainer
trainer = Trainer(model=model, args=training_arguments, train_dataset=tokenized_imdb_dataset['train'], eval_dataset=tokenized_imdb_dataset['eval'], compute_metrics=evaluation_helper)

In [26]:
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2) # To avoid retraining
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

In [ ]:
# Save model to avoid rerunning
trainer.save_model('fully-finetuned.model')

# Step 2: Fine tuning final layers only

In [28]:
adapted_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Remove gradient computation for all but classification layer
for param in adapted_model.parameters():
    param.requires_grad = False
for param in adapted_model.classifier.parameters():
    param.requires_grad = True
for param in adapted_model.pre_classifier.parameters():
    param.requires_grad = True

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# Sanity check
count_trainable_parameters(adapted_model)

592130

In [ ]:
adapted_trainer = Trainer(model=adapted_model, args=training_arguments, train_dataset=tokenized_imdb_dataset['train'], eval_dataset=tokenized_imdb_dataset['eval'], compute_metrics=evaluation_helper)

In [ ]:
adapted_trainer.train()

In [ ]:
adapted_model.save_pretrained('adapted.model')

# Step 3: Fine tuning with LoRA

In [ ]:
lora_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

### Utilities for modifying models

In [ ]:
def replace_layers(model, named_layers):
    for name, layer in named_layers.items():
        components = name.split('.')
        submodule = model
        for component in components[:-1]:
            submodule = getattr(submodule, component)
        setattr(submodule, components[-1], layer)

# From LoRA demo

In [ ]:
pretrained = torch.load('s7_pretrained.model')
pretrained

<ipython-input-33-54180ca6f566>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained = torch.load('s7_pretrained.model')


Sequential(
  (0): Linear(in_features=768, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
def batcher(batch):
    X = torch.as_tensor([x for x, _ in batch])
    Y = 1.0*torch.as_tensor([y for _, y in batch])
    return X, Y

In [ ]:
def eval_model(model):
    dl = DataLoader(list(zip(books_X_te, books_Y_te)), batch_size=32, shuffle=False, collate_fn=batcher)
    n_corr = 0
    for Xb, Yb in dl:
        with torch.no_grad():
            model_out = model(Xb)
        preds = model_out[:, 0] > 0
        gold = Yb > 0
        n_corr += sum(preds == gold).item()
    return n_corr / len(books_Y_te)

In [ ]:
eval_model(pretrained)

0.794

# Basic fine-tuning

We create a new model where we copy the weights from the pre-trained model.

In [ ]:
torch.manual_seed(0)

finetuned = nn.Sequential(
    nn.Linear(in_features=768, out_features=512),
    nn.ReLU(),
    nn.Linear(in_features=512, out_features=1)
)

# pretrained = torch.load('s7_pretrained.model')

finetuned[0].weight.data = pretrained[0].weight.data.clone()
finetuned[0].bias.data = pretrained[0].bias.data.clone()
finetuned[2].weight.data = pretrained[2].weight.data.clone()
finetuned[2].bias.data = pretrained[2].bias.data.clone()

In [ ]:
eval_model(finetuned)

0.794

In [ ]:
def train(model, n_epochs=10):
    dl = DataLoader(list(zip(books_X_tr, books_Y_tr)), batch_size=32, shuffle=True, collate_fn=batcher)

    # NOTE!
    params = [ p for p in model.parameters() if p.requires_grad_ ]

    optimizer = torch.optim.Adam(params, lr=1e-3)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(n_epochs):
        total_loss = 0
        for Xb, Yb in dl:
            model_out = model(Xb)[:, 0]
            loss = loss_fn(model_out, Yb)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        mean_loss = total_loss / len(dl)
        acc = eval_model(model)
        print(f'loss = {mean_loss:.4f}, acc = {acc:.4f}')

Your task:
- Complete `count_trainable_parameters` below.
- Count the total number of trainable parameters in the model you fine-tuned.
- Use the function `train` to fine-tune the cloned model.

# Implementing LoRA

In [ ]:
class LinearBlockWithLoRA(nn.Module):
    def __init__(self, W, r):
        """
        Initializes the LinearBlockWithLoRA.

        Args:
            W (torch.Tensor): Pre-trained weight matrix.
            r (int): Rank of the low-rank approximation.
        """
        super().__init__()

        # Store the pre-trained weight matrix
        self.W = W  # Frozen pre-trained weights

        # Get the dimensions of the pre-trained weight matrix
        out_dim, in_dim  = W.weight.shape

        # Initialize the low-rank matrices A and B
        self.A = nn.Linear(in_features=in_dim, out_features=r, bias=False)  # Low-rank adaptation A
        self.B = nn.Linear(in_features=r, out_features=out_dim, bias=False)   # Low-rank adaptation B

    def forward(self, X):
        """
        Forward pass for the LinearBlockWithLoRA.

        Args:
            X (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor after applying W and LoRA.
        """
        # Compute the output with the pre-trained weight matrix
        W_out = self.W(X)  # Using frozen weights

        # Compute the low-rank adaptation
        a_out = self.A(X) # (batch_size x in_dim) @ (in_dim x r) @ (r x out_dim)
        b_out = self.B(a_out) # (batch_size x in_dim) @ (in_dim x r) @ (r x out_dim

        # Add scaled adaptation to the pre-trained weights' output

        return W_out + b_out


Your task:
- Complete `LinearBlockWithLoRA` above
- Set up a model using this new block to replace the first linear layer. Initialize parameters from the pre-trained model. (Don't forget to switch off gradient computation for `W`.)
- Count the parameters in the new model.
- Train the new model.

In [ ]:

torch.manual_seed(0)

lora_model = nn.Sequential(
    LinearBlockWithLoRA(pretrained[0], r=8),
    nn.ReLU(),
    nn.Linear(in_features=512, out_features=1)
)

lora_model[0].W.weight.data = pretrained[0].weight.data.clone()
lora_model[0].W.bias.data = pretrained[0].bias.data.clone()
lora_model[2].weight.data = pretrained[2].weight.data.clone()
lora_model[2].bias.data = pretrained[2].bias.data.clone()

lora_model[0].W.requires_grad = False

train(lora_model, n_epochs=10)

print(count_trainable_parameters(lora_model))

loss = 0.3777, acc = 0.8060
loss = 0.3460, acc = 0.8100
loss = 0.3201, acc = 0.8180
loss = 0.3003, acc = 0.8120
loss = 0.2878, acc = 0.8220
loss = 0.2665, acc = 0.8220
loss = 0.2608, acc = 0.8280
loss = 0.2496, acc = 0.8180
loss = 0.2358, acc = 0.8140
loss = 0.2304, acc = 0.8220
404481
